In [20]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import geopandas as gpd

import contextily as ctx

import xml.etree.ElementTree as ET

In [21]:
tree = ET.parse('data/ride.tcx')
root = tree.getroot()

In [22]:
data = []
for trackpoint in root.findall('.//{http://www.garmin.com/xmlschemas/TrainingCenterDatabase/v2}Trackpoint'):
    time = trackpoint.find('{http://www.garmin.com/xmlschemas/TrainingCenterDatabase/v2}Time').text
    lat = trackpoint.find('{http://www.garmin.com/xmlschemas/TrainingCenterDatabase/v2}Position/{http://www.garmin.com/xmlschemas/TrainingCenterDatabase/v2}LatitudeDegrees').text
    lon = trackpoint.find('{http://www.garmin.com/xmlschemas/TrainingCenterDatabase/v2}Position/{http://www.garmin.com/xmlschemas/TrainingCenterDatabase/v2}LongitudeDegrees').text
    data.append([time, float(lat), float(lon)])

df = pd.DataFrame(data, columns=['Time', 'Latitude', 'Longitude'])

In [23]:
df.head()

,Time,Latitude,Longitude
0,2023-09-22T22:41:40+00:00,37.998980,-1.13315
1,2023-09-22T22:41:49+00:00,37.998233,-1.13288
2,2023-09-22T22:41:59+00:00,37.997487,-1.13261
3,2023-09-22T22:42:09+00:00,37.996740,-1.13234
4,2023-09-22T22:42:19+00:00,37.996590,-1.13301


# Calculating the distance

In [6]:
import pandas as pd
from math import radians, sin, cos, sqrt, atan2

In [7]:
def haversine(lat1, lon1, lat2, lon2):
    R = 6371
    
    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)
    
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    
    distance = R * c
    return distance

In [8]:
total_distance = 0
for i in range(1, len(df)):
    lat1, lon1 = df['Latitude'][i - 1], df['Longitude'][i - 1]
    lat2, lon2 = df['Latitude'][i], df['Longitude'][i]
    
    seg_distance = haversine(lat1, lon1, lat2, lon2)
    total_distance += seg_distance
    
f"Total Distance: {total_distance:.2f} km"

'Total Distance: 84.79 km'

# Calculating the duration

In [24]:
df.head()

,Time,Latitude,Longitude
0,2023-09-22T22:41:40+00:00,37.998980,-1.13315
1,2023-09-22T22:41:49+00:00,37.998233,-1.13288
2,2023-09-22T22:41:59+00:00,37.997487,-1.13261
3,2023-09-22T22:42:09+00:00,37.996740,-1.13234
4,2023-09-22T22:42:19+00:00,37.996590,-1.13301


In [25]:
df['Time'] = pd.to_datetime(df['Time'])

In [30]:
df = df.sort_values(by='Time')
df['TimeDiff'] = df['Time'].diff()
total_moving_time = df['TimeDiff'].sum()
total_elapsed_time = df['Time'].iloc[-1] - df['Time'].iloc[0]

display(f"Total Moving Time: {total_moving_time}")
display(f"Total Elapsed Time: {total_elapsed_time}")

'Total Moving Time: 0 days 04:04:31'

'Total Elapsed Time: 0 days 04:04:31'

In [ ]:
df.

In [ ]:
import geopandas as gpd
import plotly.express as px

# Replace 'YOUR_MAPBOX_ACCESS_TOKEN' with your actual Mapbox access token
mapbox_access_token = 'YOUR_MAPBOX_ACCESS_TOKEN'

# Create a GeoDataFrame from your DataFrame
gdf = gpd.GeoDataFrame(
    df,
    geometry=gpd.points_from_xy(df.Longitude, df.Latitude),
    crs="EPSG:4326"
)

# Create a scatter map using Plotly Express
fig = px.scatter_geo(gdf, lon="Longitude", lat="Latitude")

# Add map tiles as a background layer using Mapbox
fig.update_geos(
    projection_type="mercator",  # Use Mercator projection
    showland=True,
    landcolor="white",  # Set the land color
    center=dict(lat=0, lon=0),  # Set the initial map center
)
fig.update_layout(
    title="Interactive GeoPandas Plot with Basemap",
    mapbox=dict(
        style="satellite-streets",  # You can choose different map styles here
        center=dict(lat=0, lon=0),  # Set the initial map center
        zoom=2,  # Adjust the initial zoom level
    ),
)

# Show the plot
fig.show()
